In [ ]:
import numpy as np
import pandas as pd
import os

# Machine learning:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D, \
 MaxPooling1D, Flatten
from keras import regularizers
from keras import optimizers
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Plotting
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

import json
import talos as ta

In [ ]:
# Helper functions

def onehot_reverse(y):
    """Takes a one-hot encoded list `y` and returns a numpy array of dimension
    (m, c) (see oneshot_forward) compatible with a Keras ML algorithm."""

    m = len(y)
    c = max(y) + 1

    yf = np.zeros((m, c))

    for i, yy in enumerate(y):
      yf[i, yy] = 1

    return yf
data = []
for line in open('./Ti_O_XY.json', 'r'):
    data.append(json.loads(line))
    
print("Total of %i data points" % len(data))
print("Keys are %a" % data[0].keys())

# Using mu here not mu0
all_feature_lens = [len(data[ii]['mu']) == len(data[0]['mu'])
                    for ii in range(len(data))]
print("All features same length: %s" % np.all(np.array(all_feature_lens)))
# Only take 4, 5 and 6 coordinated:
to_ignore = [ii for ii in range(len(data)) if data[ii]['coordination'] not in [4, 5, 6]]
features = np.array([data[ii]['mu'] for ii in range(len(data)) if ii not in to_ignore])

# Minus 4 so that 6 - 4 = 2, 5 - 4 = 1 and 4 - 4 = 0; ensures that my one-hot
# decoder/encoder system can work properly!
_targets = np.array([data[ii]['coordination'] - 4 for ii in range(len(data)) if ii not in to_ignore])
targets = onehot_reverse(_targets)
print(features.shape, targets.shape)


# Generate train/validate/test splits
RANDSTATE=555
x_train, x_test, y_train, y_test = \
  train_test_split(features, targets, test_size=0.1,
                   random_state=RANDSTATE)
x_train, x_valid, y_train, y_valid = \
  train_test_split(x_train, y_train, test_size=0.1,
                   random_state=RANDSTATE)
print("training    %s ~ %s"
      % ((x_train.shape), (y_train.shape)))
print("validation  %s ~ %s"
      % ((x_valid.shape), (y_valid.shape)))
print("testing     %s ~ %s"
      % ((x_test.shape), (y_test.shape)))

# Want to normalize the targets first
y_train = y_train/np.max(y_train, axis=1, keepdims=True)
y_valid = y_valid/np.max(y_valid, axis=1, keepdims=True)
y_test = y_test/np.max(y_test, axis=1, keepdims=True)



train_feature_mean = np.mean(np.mean(x_train, axis=0))
train_feature_std = np.mean(np.std(x_train, axis=0))

valid_feature_mean = np.mean(np.mean(x_valid, axis=0))
valid_feature_std = np.mean(np.std(x_valid, axis=0))

test_feature_mean = np.mean(np.mean(x_test, axis=0))
test_feature_std = np.mean(np.std(x_test, axis=0))


print("~~~~~~~~~~~~ Mean +/- Stdev before ~~~~~~~~~~~~~~")
print("-------------------------------------------------")
print("Train features %.03f +/- %.03f"
      % (train_feature_mean, train_feature_std))
print("Valid features %.03f +/- %.03f"
      % (valid_feature_mean, valid_feature_std))
print("Test features  %.03f +/- %.03f"
      % (test_feature_mean, test_feature_std))


# Generate a feature and target scaler
feature_scaler = StandardScaler().fit(x_train)

# Utilize that scaler on the datasets
"""
x_train = feature_scaler.transform(x_train)
x_valid = feature_scaler.transform(x_valid)
x_test = feature_scaler.transform(x_test)

train_feature_mean = np.mean(np.mean(x_train, axis=0))
train_feature_std = np.mean(np.std(x_train, axis=0))

valid_feature_mean = np.mean(np.mean(x_valid, axis=0))
valid_feature_std = np.mean(np.std(x_valid, axis=0))

test_feature_mean = np.mean(np.mean(x_test, axis=0))
test_feature_std = np.mean(np.std(x_test, axis=0))

print("~~~~~~~~~~~~~ Mean +/- Stdev after ~~~~~~~~~~~~~~")
print("-------------------------------------------------")
print("Train features %.03f +/- %.03f"
      % (train_feature_mean, train_feature_std))
print("Valid features %.03f +/- %.03f"
      % (valid_feature_mean, valid_feature_std))
print("Test features  %.03f +/- %.03f"
      % (test_feature_mean, test_feature_std))
""";

def classification_model(x_train, y_train, x_val, y_val, params):

    # Params is the vessel that will carry the optimzation parameters
    dropout = params['dropout']
    act = params['activation_function']
    optimizer = params['optimizer']

    #layers = params['layers']

    model = Sequential()

    # CNN vs. MLP.
    if params['cnn']:
        x_train = np.expand_dims(x_train, axis=-1)
        x_val = np.expand_dims(x_val, axis=-1)
        model.add(Conv1D(params['kernel'],
                         params['n_filters'],
                         strides=params['strides'], padding='valid',
                         activation=act,
                         input_shape=(x_train.shape[1], 1)))
        model.add(MaxPooling1D(pool_size=params['pool_size'],
                               strides=None, padding='valid'))
        model.add(Flatten())
        model.add(Dense(params['layer0'], activation=act))
    else:
        model.add(Dense(params['layer0'], activation=act,
                        input_shape=(x_train.shape[1],)))
        model.add(Dropout(dropout))


    # note the change here relative to the regresion problem
    model.add(Dense(params['layer1'], activation=act))
    model.add(Dropout(dropout))

    model.add(Dense(params['layer2'], activation=act))
    model.add(Dropout(dropout))

    model.add(Dense(y_train.shape[1], activation='softmax'))

    model.compile(loss=params['loss_function'], optimizer=optimizer,
                metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                      batch_size=params['batch_size'],
                      epochs=params['epochs'],
                      validation_data=[x_val, y_val],
                      verbose=0,
                      shuffle=True)

    return history, model

use_params = {
    'layer0': 90,
    'layer1':60, 
    'layer2':45,
    'dropout': 0.1,
    'activation_function': 'relu',
    'optimizer': 'adam',
    'cnn': True,
    'kernel': 8,
    'n_filters': 10,
    'strides': 1,
    'pool_size': 2,
    'loss_function': 'categorical_crossentropy',
    'batch_size': 32,
    'epochs':50}

In [ ]:
_, the_model =  classification_model(x_train, y_train, x_valid, y_valid, use_params)

In [ ]:
def get_layer_output_gradient_simple_target(model,X,target_output):
    inputs = model.input
    outputs = model.output
    focus = model.output[0][target_output]
    grad = K.gradients(focus, inputs)[0]
    f = K.function([inputs], [outputs,grad])    
    return f([X])[1]

In [ ]:
#print(y_valid[:30])

four_candidate = x_valid[4]
five_candidate = x_valid[20]
#print(y_valid[fi_cnd_i])
six_candidate = x_valid[0]
#print(x_valid[0:100])
print(the_model.predict(four_candidate.reshape(1,100,1)))
print(the_model.predict(five_candidate.reshape(1,100,1)))
print(the_model.predict(six_candidate.reshape(1,100,1)))



In [ ]:
grad_44 = get_layer_output_gradient_simple_target(the_model,four_candidate.reshape(1,100,1),0).reshape(100)
grad_45 = get_layer_output_gradient_simple_target(the_model,four_candidate.reshape(1,100,1),1).reshape(100)
grad_46 = get_layer_output_gradient_simple_target(the_model,four_candidate.reshape(1,100,1),2).reshape(100)
grad_54 = get_layer_output_gradient_simple_target(the_model,five_candidate.reshape(1,100,1),0).reshape(100)
grad_55 = get_layer_output_gradient_simple_target(the_model,five_candidate.reshape(1,100,1),1).reshape(100)
grad_56 = get_layer_output_gradient_simple_target(the_model,five_candidate.reshape(1,100,1),2).reshape(100)
grad_64 = get_layer_output_gradient_simple_target(the_model,six_candidate.reshape(1,100,1),0).reshape(100)
grad_65 = get_layer_output_gradient_simple_target(the_model,six_candidate.reshape(1,100,1),1).reshape(100)
grad_66 = get_layer_output_gradient_simple_target(the_model,six_candidate.reshape(1,100,1),2).reshape(100)

In [ ]:
norm = mpl.colors.Normalize(vmin=min(grad_46), vmax=max(grad_46))
grad_colors = norm(grad_46)
cmap = mpl.cm.get_cmap('bwr')
colors = cmap(grad_colors)
for i in range(100):    
    plt.scatter(X[i],four_candidate[i],color=colors[i])    

plt.plot(X,four_candidate,zorder=-1)

In [ ]:
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib as mpl


    

##  CONFUSION MATRIX PLOT

In [ ]:
fig, ax = plt.subplots(nrows=3,ncols=3)

X = np.linspace(0,1,100)

def splash_on_plot(ax, X,Y,colors,cmap = 'bwr'):
    
    
    cmap = mpl.cm.get_cmap('bwr')
    norm = mpl.colors.Normalize(vmin=min(colors), vmax=max(colors))
    #grad_colors = norm(Y)
    #print(X.shape)
    #print(Y.shape)
    segments = []
    for i in range(len(X))[:-1]:
        segments.append([(X[i],Y[i]) , (X[i+1],Y[i+1])])
    #segments = np.column_stack([X, Y])
    
    #print(segments.shape[1])
    coll = LineCollection(segments, cmap=mpl.cm.get_cmap('bwr'),norm=norm)
    coll.set_array(colors)
    ax.add_collection(coll)
    
##ax[0][0].plot(X,four_candidate)
#ax[1][1].plot(X,five_candidate)
#ax[2][2].plot(X,six_candidate)
splash_on_plot(ax[0][0],X,four_candidate,colors=np.array(grad_44))
splash_on_plot(ax[0][1],X,four_candidate,colors=np.array(grad_45))
splash_on_plot(ax[0][2],X,four_candidate,colors=np.array(grad_46))

splash_on_plot(ax[1][0],X,five_candidate,colors=np.array(grad_54))
splash_on_plot(ax[1][1],X,five_candidate,colors=np.array(grad_55))
splash_on_plot(ax[1][2],X,five_candidate,colors=np.array(grad_56))

splash_on_plot(ax[2][0],X,six_candidate,colors=np.array(grad_64))
splash_on_plot(ax[2][1],X,six_candidate,colors=np.array(grad_65))
splash_on_plot(ax[2][2],X,six_candidate,colors=np.array(grad_66))

